# Test Load Daily

Start by loading one "new" tile.

## Imports

In [1]:
import os
import numpy as np
from astropy.io import fits
from astropy.table import Table
from sqlalchemy.exc import IntegrityError
from desiutil.log import get_logger, DEBUG
from desispec.io.meta import findfile
import specprodDB.load as db
import specprodDB.tile as t

## Initial Values

In [2]:
specprod = os.environ['SPECPROD']
# tile_id, tile_survey, tile_program = 3867, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5053, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5052, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5074, 'main', 'dark'
# tile_id, tile_survey, tile_program = 1685, 'main', 'dark'
# tile_id, tile_survey, tile_program = 40069, 'main', 'backup'
# tile_id, tile_survey, tile_program = 80950, 'sv1', 'backup'
overwrite = True

## Initialize Database

In [3]:
db.log = get_logger(DEBUG)
postgresql = db.setup_db(schema=specprod, hostname='db-loadbalancer.bweaver.development.svc.spin.nersc.org', username='desi_admin', overwrite=overwrite)
if overwrite:
    db.load_versions('computed', 'daily/v0', 'daily', specprod, 'main')

INFO:load.py:1794:setup_db: Begin creating tables.
INFO:load.py:1797:setup_db: Finished creating tables.
INFO:load.py:1540:load_versions: Loading version metadata.
INFO:load.py:1551:load_versions: Completed loading version metadata.


## Read tiles file

In [4]:
# tiles_file = findfile('tiles', readonly=True).replace('.fits', '.csv')
tiles_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'tiles-daily-patched-with-jura.csv')
tiles_table = Table.read(tiles_file, format='ascii.csv')
# tiles_table

In [9]:
row_index = np.where((tiles_table['LASTNIGHT'] >= 20201214) & (tiles_table['EFFTIME_SPEC'] > 0) & (~tiles_table['PROGRAM'].mask))[0]
tiles_table[row_index]
# row_index = np.where((tiles_table['LASTNIGHT'] >= 20201214) & (tiles_table['EFFTIME_SPEC'] > 0))[0]
# tiles_table[row_index][tiles_table[row_index]['PROGRAM'].mask]

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int64,str7,str6,str16,str19,int64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,str7,float64,int64
80713,sv1,--,m31,sv1m31,3,2700.2,10.17,41.38,0.0,758.6,771.8,1000.0,obsstart,727.8,758.6,789.8,645.0,unknown,0.9,20210110
80715,sv1,--,m31,sv1m31,3,2700.1,10.17,41.38,0.0,1906.8,1827.2,1000.0,obsend,1914.5,1906.8,2150.1,2216.6,unknown,0.9,20210115
80769,unknown,--,dithfocus,dithfocus,7,920.3,111.64,30.0,0.0,1.2,0.0,1000.0,obsstart,1.1,1.2,1.2,0.9,unknown,0.9,20210219
80816,cmx,--,dithprec,dithprec,3,540.2,99.0,66.0,0.0,14.1,45.2,1000.0,obsstart,11.4,14.1,11.6,6.8,unknown,0.9,20210223
80916,unknown,--,dithfocus,dithfocus,4,740.2,160.0,50.0,0.0,63.6,68.8,1000.0,obsstart,59.7,63.6,66.5,67.5,unknown,0.9,20210422
82026,unknown,--,dithprec,dithprec,1,200.1,251.2,50.4,0.0,73.1,59.4,1000.0,obsstart,67.5,73.1,72.7,64.4,unknown,0.9,20210423
82065,unknown,--,dithfocus,dithfocus,1,200.1,220.2,47.8,0.0,24.8,27.9,1000.0,obsstart,22.4,24.8,24.5,25.4,unknown,0.9,20210423
82052,unknown,--,dithfocus,dithfocus,1,200.1,190.0,49.5,0.0,30.1,17.3,1000.0,obsstart,25.4,30.1,26.1,14.9,unknown,0.9,20210423
81096,sv2,--,dark,sv2dark,1,1220.1,180.0,45.0,0.0,1193.2,973.8,1000.0,obsend,1111.7,1193.2,1234.4,1459.6,dark,0.9,20210429


In [6]:
tiles_table[row_index][:20]

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int64,str7,str6,str16,str19,int64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,str7,float64,int64
80615,cmx,other,m33,cmxm33,4,3600.1,24.027,31.39,0.0,3658.5,3270.6,1000.0,obsend,3582.4,3658.5,3945.2,4469.3,unknown,0.9,20201216
80609,sv1,dark,lrgqso,cmxlrgqso,15,13500.7,150.12,2.206,0.0,8202.4,8317.8,4000.0,obsend,8103.8,8202.4,9070.2,12421.2,dark,0.9,20201217
80606,sv1,dark,elg,cmxelg,12,10800.7,36.448,-4.501,0.0,7179.2,7183.1,4000.0,obsend,6967.0,7179.2,7653.2,8686.3,dark,0.9,20201219
80620,sv1,dark,lrgqso,sv1lrgqso,13,11700.7,144.0,65.0,0.0,8228.4,7516.2,4000.0,obsend,7522.7,8228.4,7875.7,7957.3,dark,0.9,20201222
80611,sv1,bright,bgsmws,sv1bgsmws,13,3620.9,35.22,30.3,0.0,691.2,712.8,1000.0,obsstart,660.5,754.1,691.2,440.5,bright,0.9,20201222
80607,sv1,dark,lrgqso,cmxlrgqso,77,18804.9,0.0,0.0,0.0,10817.0,0.0,1000.0,obsend,9949.4,10817.0,10569.1,10520.4,dark,0.9,20201222
80612,sv1,bright,bgsmws,sv1bgsmws,50,5658.9,66.68,-5.24,0.0,1611.0,1572.8,1000.0,obsend,1522.9,1658.4,1611.0,904.2,bright,0.9,20201223
80608,sv1,dark,elg,cmxelg,27,24301.6,106.74,56.2,0.0,17113.6,14302.2,4000.0,obsend,15770.5,17113.6,16838.1,17001.9,dark,0.9,20201223
80623,sv1,dark,elg,sv1elg,10,9000.5,155.0,32.425,0.0,5560.0,5750.7,4000.0,obsend,5392.1,5560.0,6032.1,8957.2,dark,0.9,20201223


In [7]:
load_tiles = db.Tile.convert(tiles_table, row_index=row_index)

In [8]:
load_tiles[:20]

[Tile(tileid=80615),
 Tile(tileid=80609),
 Tile(tileid=80606),
 Tile(tileid=80620),
 Tile(tileid=80611),
 Tile(tileid=80607),
 Tile(tileid=80612),
 Tile(tileid=80608),
 Tile(tileid=80623),
 Tile(tileid=80616),
 Tile(tileid=80617),
 Tile(tileid=80622),
 Tile(tileid=80629),
 Tile(tileid=80638),
 Tile(tileid=80635),
 Tile(tileid=80641),
 Tile(tileid=80642),
 Tile(tileid=80633),
 Tile(tileid=80640),
 Tile(tileid=80676)]

In [10]:
try:
    db.dbSession.add_all(load_tiles)
    db.dbSession.commit()
except IntegrityError:
    print("ERROR: Some tiles are already loaded!")
    db.dbSession.rollback()

## Read exposures file

The daily exposures file may contain exposures with `EFFTIME_SPEC == 0`. We do not want to load these.

In [11]:
# exposures_file = findfile('exposures', readonly=True)
exposures_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'exposures-daily-patched-with-jura.fits')
exposures_table = Table.read(exposures_file, format='fits', hdu='EXPOSURES')
frames_table = Table.read(exposures_file, format='fits', hdu='FRAMES')
# exposures_table[exposures_table['TILEID'] == new_tile.tileid]

In [12]:
load_exposures = list()
for new_tile in load_tiles:
    row_index = np.where((exposures_table['TILEID'] == new_tile.tileid) & (exposures_table['EFFTIME_SPEC'] > 0))[0]
    if len(row_index) > 0:
        # load_exposures += db.Exposure.convert(exposures_table, row_index=row_index)
        if (exposures_table[row_index]['MJD'] < 50000).any():
            print("WARNING: Invalid MJD values detected for tile {0:d}!".format(new_tile.tileid))
            bad_index = np.where((exposures_table['TILEID'] == new_tile.tileid) & (exposures_table['EFFTIME_SPEC'] > 0) & (exposures_table['MJD'] < 50000))[0]
            for row in exposures_table[['EXPID', 'NIGHT', 'MJD', 'EFFTIME_SPEC']][bad_index]:
                raw_data_file = os.path.join(os.environ['DESI_SPECTRO_DATA'], "{0:08d}".format(row['NIGHT']), "{0:08d}".format(row['EXPID']), "desi-{0:08d}.fits.fz".format(row['EXPID']))
                with fits.open(raw_data_file, mode='readonly') as hdulist:
                    mjd_obs = hdulist['SPEC'].header['MJD-OBS']
                    print("WARNING: tile {0:d} exposure {1:d} has MJD-OBS = {2:f} in {3}!".format(new_tile.tileid, row['EXPID'], mjd_obs, raw_data_file))
    else:
        print("ERROR: No valid exposures found for tile {0:d}, even though EFFTIME_SPEC == {1:f}!".format(new_tile.tileid, new_tile.efftime_spec))
        bad_index = np.where((exposures_table['TILEID'] == new_tile.tileid))[0]
        print(exposures_table[['EXPID', 'NIGHT', 'MJD', 'EFFTIME_SPEC']][bad_index])
# load_frames = list()
# for exposure in load_exposures:
#     row_index = np.where(frames_table['EXPID'] == exposure.expid)[0]
#     assert len(row_index) > 0
#     load_frames += db.Frame.convert(frames_table, row_index=row_index)
# load_exposures

ERROR: No valid exposures found for tile 81088, even though EFFTIME_SPEC == 24.800000!
EXPID  NIGHT   MJD EFFTIME_SPEC
----- -------- --- ------------
83416 20210404 0.0          0.0
ERROR: No valid exposures found for tile 158, even though EFFTIME_SPEC == 60.500000!
EXPID  NIGHT   MJD EFFTIME_SPEC
----- -------- --- ------------
83543 20210405 0.0          0.0
ERROR: No valid exposures found for tile 23, even though EFFTIME_SPEC == 66.700000!
EXPID  NIGHT   MJD EFFTIME_SPEC
----- -------- --- ------------
83524 20210405 0.0          0.0
ERROR: No valid exposures found for tile 50, even though EFFTIME_SPEC == 73.800000!
EXPID  NIGHT   MJD EFFTIME_SPEC
----- -------- --- ------------
83525 20210405 0.0          0.0
ERROR: No valid exposures found for tile 131, even though EFFTIME_SPEC == 79.300000!
EXPID  NIGHT   MJD EFFTIME_SPEC
----- -------- --- ------------
83732 20210406 0.0          0.0
ERROR: No valid exposures found for tile 185, even though EFFTIME_SPEC == 49.500000!
EXPID  NIG

In [ ]:
# db.dbSession.rollback()
db.dbSession.add_all(load_exposures)
db.dbSession.commit()

In [ ]:
# db.dbSession.rollback()
db.dbSession.add_all(load_frames)
db.dbSession.commit()

## Load photometry for the tile

When tractor photometry is written out by John Moustakas' VAC code, only objects with `brickname != ''` are written.

In [ ]:
potential_targets_table = t.potential_targets(load_tiles[1].tileid)
# potential_targets_table

In [ ]:
potential_cat = t.potential_photometry(load_tiles[1], potential_targets_table)
# potential_cat

In [ ]:
potential_targetphot = t.targetphot(potential_cat)
potential_tractorphot = t.tractorphot(potential_cat)

In [ ]:
load_photometry = t.load_photometry(potential_tractorphot)

### Load photometry, such as it is, for objects that are not in the tractor catalog

In [ ]:
load_targetphot = t.load_targetphot(potential_targetphot, load_photometry)

### Load the target table

In [ ]:
load_target = t.load_target(load_tiles[1], potential_targetphot)

## Load tile/cumulative redshifts

Need a way to compute "best" spectra as new tiles are added. There are a lot of columns that come from other sources here, so need to track these down.

In [ ]:
load_ztile = t.load_redshift(load_tiles[0])

## Load fiberassign and potential

In [ ]:
load_fiberassign, load_potential = t.load_fiberassign(load_tiles[0])

## Recompute global values

The global values are the primary classification and number of spectra.

In [ ]:
t.update_primary()

## q3c Update

`tile`, `exposure`, `photometry`, `fiberassign`

In [ ]:
t.update_q3c()